In [31]:
from glob import glob
from tqdm import tqdm
import json
import re

In [32]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [34]:
files = sorted(glob('code-instructions/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

112329

In [35]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

In [38]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
]

In [39]:
count, indon_count = 0, 0

indon_texts = []
for f in tqdm(files):
    try:
        with open(f) as fopen:
            data = json.load(fopen)
    except:
        continue

    if data['output'] is None:
        continue
        
    if any([r in data['instruction'].lower() for r in rejected_instructions]):
        continue

    indon_texts.append(data['instruction'])
    indon_texts.append(data['output'])

    count += 1

100%|█████████████████████████████████| 112329/112329 [00:13<00:00, 8535.17it/s]


In [40]:
len(indon_texts)

212658

In [41]:
print(indon_texts[1])

Untuk menggantikan ayat "Tom tiba awal ke majlis" dengan menggunakan koma keduanya dan kata hubung dalam kod Go, dapat menggunakan frasa "Tom, yang juga tiba awal, ke majlis".


In [42]:
with open('code-instructions.texts', 'w') as fopen:
    for t in set(indon_texts):
        fopen.write(f'{json.dumps(t)}\n')

In [43]:
!cp code-instructions.texts ~/ssd3/ctranslate2

In [45]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/code-instructions.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
        
len(mapping)

211739

In [46]:
rejected = 0

with open('synthetic-code-instructions.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
            
        if data['output'] is None:
            continue
        
        output_ms = mapping.get(data['output'])
        instruction_ms = mapping.get(data['instruction'])
        
        indon_ins = False
        if check_indon(data['instruction']):
            indon_ins = True
        
        indon_output = False
        if check_indon(data['output']):
            indon_output = True
        
        rejected_ins = False
        if 1024 <= len(enc.encode(data['instruction'])) <= 1025:
            rejected_ins = True
            
        if any([r in data['instruction'].lower() for r in rejected_instructions]):
            rejected_ins = True
        
        rejected_output = False
        if 1024 <= len(enc.encode(data['output'])) <= 1025:
            rejected_output = True
            
        if rejected_ins or rejected_output:
            rejected += 1
            
        data['indon_ins'] = indon_ins
        data['indon_output'] = indon_output
        data['instruction_ms'] = instruction_ms
        data['output_ms'] = output_ms
        data['rejected_ins'] = rejected_ins
        data['rejected_output'] = rejected_output
        
        fopen_l.write(f'{json.dumps(data)}\n')
        count += 1

100%|█████████████████████████████████| 112329/112329 [00:49<00:00, 2254.05it/s]
